In [1]:
import pandas as pd
solar_installs = pd.read_csv('Data/Solar_Footprints_V2_7811899327930675815.csv')

In [2]:
solar_installs.sort_values(by='Shape__Area', ascending=False)

,OBJECTID,County,Acres,Install Type,Urban or Rural,Combined Class,Distance to Substation (Miles) GTET 100 Max Voltage,Percentile (GTET 100 Max Voltage),Substation Name GTET 100 Max Voltage,HIFLD ID (GTET 100 Max Voltage),...,Percentile (GTET 200 Max Voltage),Substation Name GTET 200 Max Voltage,HIFLD ID (GTET 200 Max Voltage),Distance to Substation (Miles) CAISO,Percentile (CAISO),Substation CASIO Name,HIFLD ID (CAISO),Solar Technoeconomic Intersection,Shape__Area,Shape__Length
2771,2772,Riverside County,7438.603255,Ground,Rural,Ground - Rural,3.886829,50th to 75th,Blythe Walc,304454.0,...,50th to 75th,Colorado River,303761.0,6.290021,50th to 75th,Colorado River,303761.0,Outside,3.010313e+07,35928.275147
5387,5388,Kings County,5058.074647,Ground,Rural,Ground - Rural,2.932513,50th to 75th,Unknown,309129.0,...,0 to 25th,Unknown,309129.0,3.605828,25th to 50th,Henrietta,304664.0,Outside,2.046914e+07,21487.606885
2804,2805,Riverside County,3875.067318,Ground,Rural,Ground - Rural,2.513474,25th to 50th,Unknown,300795.0,...,0 to 25th,Unknown,300795.0,4.587985,25th to 50th,Kem,306318.0,Outside,1.568186e+07,22390.045184
3339,3340,San Bernardino County,3444.751330,Ground,Rural,Ground - Rural,30.154816,75th to 100th,Sandy Valley,NaN,...,75th to 100th,Camino,305759.0,0.000000,0 to 25th,Ivanpah,310135.0,Outside,1.394046e+07,23319.291206
1362,1363,Kings County,2894.855803,Ground,Rural,Ground - Rural,1.486667,25th to 50th,Unknown,309129.0,...,0 to 25th,Unknown,309129.0,2.166880,0 to 25th,Henrietta,304664.0,Outside,1.171514e+07,22346.529151
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4278,4279,Santa Clara County,0.034271,Rooftop,Urban,Rooftop - Urban,0.876115,0 to 25th,Ames Distribution,310434.0,...,25th to 50th,Ravenswood,308197.0,7.727228,50th to 75th,Scott,300561.0,Within,1.386912e+02,53.207933
5160,5161,Kern County,0.029450,Ground,Rural,Ground - Rural,6.904726,75th to 100th,Whirlwind,306225.0,...,50th to 75th,Whirlwind,306225.0,6.904726,50th to 75th,Whirlwind,306225.0,Outside,1.191793e+02,49.438403
5166,5167,Kern County,0.017946,Ground,Rural,Ground - Rural,6.659845,75th to 100th,Whirlwind,306225.0,...,50th to 75th,Whirlwind,306225.0,6.659845,50th to 75th,Whirlwind,306225.0,Outside,7.262562e+01,34.145689
5167,5168,Kern County,0.007517,Ground,Rural,Ground - Rural,6.266930,75th to 100th,Whirlwind,306225.0,...,50th to 75th,Whirlwind,306225.0,6.266930,50th to 75th,Whirlwind,306225.0,Outside,3.042128e+01,158.366524


In [17]:
solar_installs.groupby('County')['Acres'].sum().sort_values(ascending=False)

County
Kern County               31120.012374
Riverside County          27407.234213
Imperial County           16433.794820
San Bernardino County     16015.421073
Kings County              15900.618199
Los Angeles County        11084.963558
Fresno County              8462.816938
San Luis Obispo County     4749.201895
Tulare County              3302.904574
Merced County              2436.533156
Monterey County            1835.785914
Sacramento County          1770.111219
Stanislaus County          1653.878766
San Benito County          1228.380798
San Diego County           1191.473327
Madera County               802.517314
Santa Clara County          505.015307
Santa Barbara County        485.401797
Orange County               470.806142
San Joaquin County          456.849181
Contra Costa County         425.123837
Alameda County              407.018556
Solano County               361.158083
Ventura County              248.286281
Yolo County                 217.161747
Sonoma County     

In [ ]:
import requests
import pandas as pd
import urllib.parse
import time

API_KEY = "{{YOUR_API_KEY}}"
EMAIL = "shabari@umich.edu"
BASE_URL = "https://developer.nrel.gov/api/nsrdb/v2/solar/nsrdb-GOES-full-disc-v4-0-0-download.json?"
POINTS = [
'736277'
]

def main():
    input_data = {
        'attributes': 'air_temperature,alpha,aod,asymmetry,clearsky_dhi,clearsky_dni,clearsky_ghi,cloud_fill_flag,cloud_type,dew_point,dhi,dni,fill_flag,ghi,ozone,relative_humidity,solar_zenith_angle,ssa,surface_albedo,surface_pressure,total_precipitable_water,wind_direction,wind_speed',
        'interval': '10',
        'to_utc': 'false',
        
        'api_key': API_KEY,
        'email': EMAIL,
    }
    for name in ['2023']:
        print(f"Processing name: {name}")
        for id, location_ids in enumerate(POINTS):
            input_data['names'] = [name]
            input_data['location_ids'] = location_ids
            print(f'Making request for point group {id + 1} of {len(POINTS)}...')

            if '.csv' in BASE_URL:
                url = BASE_URL + urllib.parse.urlencode(data, True)
                # Note: CSV format is only supported for single point requests
                # Suggest that you might append to a larger data frame
                data = pd.read_csv(url)
                print(f'Response data (you should replace this print statement with your processing): {data}')
                # You can use the following code to write it to a file
                # data.to_csv('SingleBigDataPoint.csv')
            else:
                headers = {
                  'x-api-key': API_KEY
                }
                data = get_response_json_and_handle_errors(requests.post(BASE_URL, input_data, headers=headers))
                download_url = data['outputs']['downloadUrl']
                # You can do with what you will the download url
                print(data['outputs']['message'])
                print(f"Data can be downloaded from this url when ready: {download_url}")

                # Delay for 1 second to prevent rate limiting
                time.sleep(1)
            print(f'Processed')


def get_response_json_and_handle_errors(response: requests.Response) -> dict:
    """Takes the given response and handles any errors, along with providing
    the resulting json

    Parameters
    ----------
    response : requests.Response
        The response object

    Returns
    -------
    dict
        The resulting json
    """
    if response.status_code != 200:
        print(f"An error has occurred with the server or the request. The request response code/status: {response.status_code} {response.reason}")
        print(f"The response body: {response.text}")
        exit(1)

    try:
        response_json = response.json()
    except:
        print(f"The response couldn't be parsed as JSON, likely an issue with the server, here is the text: {response.text}")
        exit(1)

    if len(response_json['errors']) > 0:
        errors = '\n'.join(response_json['errors'])
        print(f"The request errored out, here are the errors: {errors}")
        exit(1)
    return response_json

if __name__ == "__main__":
    main()